# Run preprocessing


## CSV to pickle
```bash
python -m src.preprocessing.csv2pkl --input_dir D:/DTU/AIS-MDA/data/raw/ --output_dir D:/DTU/AIS-MDA/data/processed_pickle/
```


## Map_reduce
```bash
python -m src.preprocessing.map_reduce --input_dir  D:/DTU/AIS-MDA/data/processed_pickle/ --temp_dir TEMP_DIR --final_dir D:/DTU/AIS-MDA/data/map_reduced
```


## Train test split
```bash
python -m src.preprocessing.train_test_split --data_dir D:/DTU/AIS-MDA/data/map_reduced/ --val_size 0.1 --test_size 0.1 --random_state 42
```


## Train Model
```bash
python -m src.train.train_traj --config configs/test_alex.yaml
```



# VAL new
```bash
python -m src.eval.evaluate_traj_new \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --plot
```

# TEST

```bash
python -m src.eval.evaluate_traj_new \
  --split_dir data/map_reduced/test \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans
```
# out
metrics/traj_tptrans_test.json






# new abs
python -m src.eval.eval_traj_abs \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --max_plots 8 \
  --out_dir data/figures

# new traj eval
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 0 --lon_idx 1 \
  --past_tail 256 \
  --max_plots 8 \
  --out_dir data/figures

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 1 --lon_idx 0 \
  --past_len 64 --max_plots 8 \
  --out_dir data/figures










# old
python -m src.eval.eval_traj_newnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 1 --lon_idx 0 \
  --max_plots 8 \
  --out_dir data/figures \
  --viz_match_lengths










# Evaluate on Map-Reduce split
```bash
python -m src.eval.evaluate_traj_new \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --plot
```

```bash
python -m src.eval.evaluate_traj_new \
  --split_dir data/map_reduced/test \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans
```













# Validate new new
```bash
python -m src.eval.eval_traj_newnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 1 --lon_idx 0 \   # adjust if your feature order differs
  --max_plots 6 \
  --out_dir data/figures
```

python -m src.eval.eval_traj_newnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 1 --lon_idx 0 \   # adjust if your feature order differs
  --out_dir data/figures





python -m src.eval.eval_traj_newnew `
  --split_dir data\map_reduced\val `
  --ckpt data\checkpoints\traj_tptrans.pt `
  --model tptrans `
  --lat_idx 1 `
  --lon_idx 0 `
  --max_plots 8 `
  --out_dir data\figures

python -m src.eval.eval_traj_newnew \
  --split_dir "data/map_reduced/val" \
  --ckpt "data/checkpoints/traj_tptrans.pt" \
  --model tptrans \
  --lat_idx 1 \
  --lon_idx 0 \
  --max_plots 8 \
  --out_dir "data/figures"



python -m src.eval.eval_traj_newnew \
  --split_dir "data/map_reduced/val" \
  --ckpt "data/checkpoints/traj_tptrans.pt" \
  --model tptrans \
  --lat_idx 1 \
  --lon_idx 0 \
  --max_plots 8 \
  --out_dir "data/figures"






python -m src.eval.eval_traj_abs \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --lat_idx 1 --lon_idx 0 \
  --max_plots 8 \
  --out_dir data/figures








### Unsupervised Pretraining MSP

1) Unsupervised pretraining

```bash
python -m src.train.pretrain_msp_new \
  --split_dir data/map_reduced/train \
  --out data/checkpoints/pretrain_msp.pt \
  --epochs 5
```

2) Fine-tune supervised

in configs/test.yaml add:
warm_start_msp: data/checkpoints/pretrain_msp.pt

```bash
python -m src.train.train_traj_unsup --config configs/test_unsub.yaml
```





In [1]:
import numpy as np, json, pathlib
p = pathlib.Path("data/map_reduced/val")
print((p/"scaler.npz").exists())
S = np.load(p/"scaler.npz")
print(S["mean"].shape, S["std"].shape)


False


FileNotFoundError: [Errno 2] No such file or directory: 'data\\map_reduced\\val\\scaler.npz'

In [4]:
import os
os.chdir(r"D:\DTU\AIS-MDA")
print("cwd:", os.getcwd())

cwd: D:\DTU\AIS-MDA


In [5]:
import os 
from src.utils.datasets import AISDataset
ds = AISDataset("data/map_reduced/val", max_seqlen=96)
x,y = ds[0]; print(x.shape, y.shape)      # e.g. [T,F], [H,2]
print(y[:3])  # should look like small step deltas, not a long ramp


Filtering trajectories...
Valid samples: 77/87
torch.Size([64, 4]) torch.Size([12, 2])
tensor([[0.0905, 0.4064],
        [0.0920, 0.4077],
        [0.0924, 0.4092]])


In [6]:
import pickle, numpy as np
d = pickle.load(open("data/map_reduced/val/211286440_0_processed.pkl","rb"))
x, y = d["traj"]["x"], d["traj"]["y"]   # or however you stored it
print(x.shape, y.shape)                 # expect [T,F], [H,2]
print("y first 5:", y[:5])


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices